In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import keras
from sklearn.metrics import classification_report,confusion_matrix
import nltk
import matplotlib.pyplot as plt
from sklearn.model_selection import (train_test_split,cross_val_predict,
                                     cross_val_score,cross_validate,GridSearchCV,StratifiedKFold
                                    )
from sklearn import ensemble, linear_model,svm,neighbors,neural_network
from sklearn.metrics import (classification_report,confusion_matrix,
                             accuracy_score,auc,mean_absolute_error,r2_score,mean_squared_error,
                             plot_roc_curve,roc_auc_score,roc_curve)
from sklearn.preprocessing import (OneHotEncoder,LabelEncoder,MinMaxScaler,StandardScaler,)
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier,XGBRegressor
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

In [3]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer(language='english')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = {"a","an","the"}
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def remove_html_tags(raw_html):
  return re.sub(r'<.*?>', '', raw_html)

def remove_urls(raw_text):
    raw_text = re.sub(r'http\S+', '', raw_text)
    return re.sub(r'www\S+', '', raw_text)

def remove_nonalpha(raw_text):
  return re.sub(r'[^a-zA-Z\s]', '', raw_text)

def remove_lines(raw_text):
  return re.sub(r'\n',' ',raw_text)

def lemme_stem_stop(raw_text):
  words = word_tokenize(raw_text)
  final_words=[]
  for word in words:
    word = word.lower()
    if word not in stop_words:
      word = snowball_stemmer.stem(word)
      final_words.append(word)
  return " ".join(final_words)

def pre_processing(raw_text):
  return lemme_stem_stop(remove_lines(remove_nonalpha(remove_urls(remove_html_tags(raw_text)))))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def model(rmf,x,y,isval=False):
    if not isval:
        rmf.fit(x,y)
    y_pred=rmf.predict(x)
    if isval:
        print("\nValidation:")
    else:
        print("\nTrain:")
    print(classification_report(y,y_pred))
    print()
    print(confusion_matrix(y,y_pred))
    print("-"*50)
    return rmf

In [ ]:
text_df = pd.read_csv("/content/Train_and_validation.csv")
text_df

,Unnamed: 0,Unnamed: 0.1,id,text,bias
0,0,0,1,Besides these forests naturally developed fore...,0
1,1,1,2,Your mother cleans the rice by separating husk...,0
2,2,2,3,All free gifts of nature and all things which ...,0
3,3,3,4,Humanism - the concept that man is the measure...,0
4,4,4,5,Various designs in clothing are found dependin...,0
...,...,...,...,...,...
1147,1991,1991,1992,"Mulberry , soyabean , cotton , tobacco , flax ...",1
1148,1993,1993,1994,b. Self Employment : Picture : . To eradicate ...,1
1149,1995,1995,1996,Bring both the ends of threads together and pu...,1
1150,1997,1997,1998,PotassiumProtein synthesisEnables the opening ...,1


In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer() #Change Vectorizer here
X = text_df["text"]
y = text_df["bias"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42,stratify=y)

In [ ]:
X_train.shape

(1036, 5041)

In [ ]:
import tensorflow as tf

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN,Embedding
from keras.preprocessing import sequence, text
from keras.utils.data_utils import pad_sequences


In [ ]:
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(X_train) + list(X_test))
xtrain_seq = token.texts_to_sequences(X_train)
xvalid_seq = token.texts_to_sequences(X_test)

#zero pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
# # Prepare the input X and target Y
# def get_XY(dat, time_steps):
#     # Indices of target array
#     Y_ind = np.arange(time_steps, len(dat), time_steps)
#     Y = dat[Y_ind]
#     # Prepare X
#     rows_x = len(Y)
#     X = dat[range(time_steps*rows_x)]
#     X = np.reshape(X, (rows_x, time_steps, 1))    
#     return X, Y

In [ ]:
# A simpleRNN without any pretrained embeddings and one dense layer
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     100,
                     input_length=1500))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
# model.summary()

In [ ]:
# rmf=tf.keras.layers.SimpleRNN(
#     units=4,
#     activation="tanh",
#     use_bias=True,
#     kernel_initializer="glorot_uniform",
#     recurrent_initializer="orthogonal",
#     bias_initializer="zeros",
#     kernel_regularizer=None,
#     recurrent_regularizer=None,
#     bias_regularizer=None,
#     activity_regularizer=None,
#     kernel_constraint=None,
#     recurrent_constraint=None,
#     bias_constraint=None,
#     dropout=0.0,
#     recurrent_dropout=0.0,
#     return_sequences=False,
#     return_state=False,
#     go_backwards=False,
#     stateful=False,
#     unroll=False,
# )

In [ ]:
# from sklearn.metrics import mean_squared_error

In [ ]:
# def create_RNN( dense_units, activation):
#     model = Sequential()
#     model.add(rmf)
#     model.add(Dense(units=dense_units, activation=activation[1]))
#     model.add(Dense(units=1,activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam')
#     return model

In [ ]:
# def print_error(trainY, testY, train_predict, test_predict):    
#     # Error of predictions
#     train_rmse = math.sqrt(mean_squared_error(trainY, train_predict))
#     test_rmse = math.sqrt(mean_squared_error(testY, test_predict))
#     # Print RMSE
#     print('Train RMSE: %.3f RMSE' % (train_rmse))
#     print('Test RMSE: %.3f RMSE' % (test_rmse))    

In [ ]:
# time_steps=1

In [ ]:
# model = create_RNN(dense_units=4, 
#                    activation=['tanh', 'tanh'])
model.fit(xtrain_pad, y_train, epochs=5, batch_size=64) #Multiplying by Strategy to run on TPU's


Epoch 1/5
17/17 [==============================] - 17s 906ms/step - loss: 0.6934 - accuracy: 0.5483
Epoch 2/5
17/17 [==============================] - 13s 737ms/step - loss: 0.5667 - accuracy: 0.8272
Epoch 3/5
17/17 [==============================] - 13s 737ms/step - loss: 0.4106 - accuracy: 0.9131
Epoch 4/5
17/17 [==============================] - 13s 768ms/step - loss: 0.2617 - accuracy: 0.9942
Epoch 5/5
17/17 [==============================] - 12s 732ms/step - loss: 0.1241 - accuracy: 0.9971


In [1]:
from sklearn import metrics

In [ ]:
model.evaluate(xvalid_pad,y_test)

4/4 [==============================] - 1s 136ms/step - loss: 0.6337 - accuracy: 0.6466


[0.6336637139320374, 0.6465517282485962]